# ERA5


## Introduction

Exemple de calcul distribué sur le dataset ERA5 avec le stack de Pangeo

## Environnement

In [32]:
# Display output of plots directly in Notebook
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import intake
import numpy as np
import pandas as pd
import xarray as xr
import hvplot.xarray
import xshape
import s3fs
import pandas as pd
import geopandas as gpd
import hydrotools as ht

## Dask Distributed Cluster

In [2]:
# Create cluster
from dask_gateway import Gateway
from dask.distributed import Client
# gateway = Gateway()
# cluster = gateway.new_cluster()
# cluster.adapt(minimum=2, maximum=100)
# # Connect to cluster
# client = Client(cluster)
client = Client()

# Display cluster dashboard URL
# cluster

client

Client Scheduler: tcp://127.0.0.1:32907 Dashboard: http://127.0.0.1:33303/status,Cluster Workers: 4 Cores: 12 Memory: 33.48 GB



☝️ Link to scheduler dashboard will appear above.

In [18]:
lst_shp = ['s3://watersheds-polygons/MELCC/json/011201/011201.json',
           's3://watersheds-polygons/MELCC/json/050127/050127.json']

client_kwargs={"endpoint_url":"https://s3.us-east-2.wasabisys.com"}

In [19]:
s3 = s3fs.S3FileSystem(client_kwargs=client_kwargs,
                               anon=True)  # public read
gdf = pd.concat([gpd.GeoDataFrame.from_file(s3.open(file), encoding='latin-1')
                 for file in lst_shp])

In [21]:
gdf

,Station,Superficie,geometry
0,011201,1134.766897644438,"POLYGON ((-66.60283 48.52781, -66.60283 48.527..."
0,050127,28298.18417805583,"POLYGON ((-72.90416 47.61491, -72.90415 47.614..."


In [27]:
import hvplot.pandas
gdf.hvplot(geo=True, 
           tiles='ESRI',
           xlim=(-60,-80),
           width=900,
           height=400, 
           color='y',
           hover_cols=['Station','Superficie'])

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]   (Station,Superficie)

## Load data into xarray from a catalog using intake-esm


In [6]:
import intake
cat_url = 'https://raw.githubusercontent.com/hydrologie/datasets/main/intake-catalogs/atmosphere.yml'
cat = intake.open_catalog(cat_url)

In [28]:
import xarray as xr
import fsspec
bucket_era5 ='s3://era5-atlantic-northeast/zarr/single-levels/reanalysis'


store = fsspec.get_mapper(url=bucket_era5,
                          client_kwargs=client_kwargs,
                          anon=True)

# Ouverture du zarr vers dataset (xarray)
ds = xr.open_zarr(store,
                       consolidated=True,
                       chunks='auto')
ds

,Array,Chunk
Bytes,47.33 GB,44.64 MB
Shape,"(359400, 93, 177)","(8928, 25, 25)"
Count,1313 Tasks,1312 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,47.33 GB,44.64 MB
Shape,"(359400, 93, 177)","(8928, 25, 25)"
Count,1313 Tasks,1312 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [33]:
df_era5 = ht.clip_polygon_to_dataframe(dataset=ds,
                                       geodataframe=gdf,
                                       geodf_index_column='Station',
                                       variable='tp',  
                                       aggregation='sum',
                                       resample_time='1D', # originalement au pas de temps horaire
                                       from_tz='UTC',
                                       to_tz='America/Montreal',
                                       latlng_names=['latitude',
                                                     'longitude']
                                       )

011201
050127


In [34]:
import hvplot.pandas

In [43]:
df_era5['year'] = df_era5.index.year
df_era5.hvplot(x='time', groupby='year', widget_location='top', width=900, grid=True)

Column
    [0] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] DiscreteSlider(formatter='%d', margin=(20, 20, 20, 20), name='year', options=OrderedDict([('1978', ...]), value=1978, value_throttled=1978, width=250)
        [2] HSpacer()
    [1] HoloViews(DynamicMap, widget_location='top')

,011201,050127
time,,
1978-12-31 00:00:00-05:00,0.000000,0.000000
1979-01-01 00:00:00-05:00,0.016225,0.015870
1979-01-02 00:00:00-05:00,0.015041,0.002905
1979-01-03 00:00:00-05:00,0.026687,0.000532
1979-01-04 00:00:00-05:00,0.000821,0.000983
...,...,...
2019-12-27 00:00:00-05:00,0.004536,0.003057
2019-12-28 00:00:00-05:00,0.000553,0.000835
2019-12-29 00:00:00-05:00,0.000572,0.000144


Int64Index([1978, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979,
            ...
            2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019],
           dtype='int64', name='time', length=14976)